In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from datetime import datetime

In [9]:
import logging
# ロギングの設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [10]:
# スクレイピング対象のURL
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC005/?fw2=&mt=9999999&cn=9999999&ta=13&et=9999999&sc=13104&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&srch_navi=1&mb=0&shkr4=03&cb=0.0'

# requestsでURLからデータを取得
response = requests.get(url)
response.encoding = response.apparent_encoding

# BeautifulSoupオブジェクトの生成
soup = BeautifulSoup(response.text, 'html.parser')


In [11]:
# 物件情報を格納するためのリスト
properties_list = []

# 空のデータフレームを作成
df = pd.DataFrame()

# 物件情報が含まれる要素をすべて取得
properties = soup.find_all('div', class_='property')

In [12]:
# 物件情報の抽出
for prop in properties:
    # 物件名が記載されているclassを選択
    title = prop.find('h2', class_='property_inner-title').text.strip()
    property_link = prop.find('a', class_='js-cassetLinkHref')['href']

    # 取り扱い店舗が記載されているclassを選択
    property_stores = prop.find('a', class_='js-noCassetteLink').text.strip()
    property_stores_link = prop.find('a', class_='js-noCassetteLink')['href']

    # 住所・アクセス経路・賃料・管理費・間取り・専有面積・向き・築年数・マンションかアパート・敷金・礼金が記載されているtdタグを選択
    detailbox = prop.find('div', class_='detailbox')
    if detailbox:

        # アクセス経路が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        access_element = prop.find('div', class_='detailnote-box').text.strip()

        access_elements = access_element.split('\n')
        access_1 = access_elements[0].strip() if len(access_elements) > 0 else None
        access_2 = access_elements[1].strip() if len(access_elements) > 1 else None
        access_3 = access_elements[2].strip() if len(access_elements) > 2 else None

        # 賃料・管理費が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        fee_element = detailbox.find_all('td', class_='detailbox-property-col')[0].text.strip()
        rent_price = fee_element.split('\n')[0].strip()
        management_fee =  fee_element.split('\n')[1].strip()

        # 敷金・礼金が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        security_deposit_element = detailbox.find_all('td', class_='detailbox-property-col')[1].text.strip()
        security_deposit = security_deposit_element.split('\n')[0].strip()
        key_money = security_deposit_element.split('\n')[1].strip()

        # 間取り・専有面積・向きが記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        house_layout_element = detailbox.find_all('td', class_='detailbox-property-col')[2].text.strip()
        house_layout =  house_layout_element.split('\n')[0].strip()
        exclusive_area =  house_layout_element.split('\n')[2].strip()
        direction =  house_layout_element.split('\n')[4].strip()

        # 築年数とマンションかアパートが記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        building_type_element = detailbox.find_all('td', class_='detailbox-property-col')[3].text.strip()
        building_type =  building_type_element.split('\n')[0].strip()
        building_age = building_type_element.split('\n')[2].strip()

        # 住所が記載されているtdタグを正確に選択
        address = detailbox.find_all('td', class_='detailbox-property-col')[4].text.strip()

    # データリストに情報を追加
    properties_list.append({
        '物件名': title,
        'URL': "https://suumo.jp"+property_link,
        '住所': address,
        'アクセス1': access_1,
        'アクセス2': access_2,
        'アクセス3': access_3,
        '築年数': building_age,
        '賃貸料': rent_price,
        '管理費': management_fee,
        '間取り': house_layout,
        '専有面積': exclusive_area,        
        '向き': direction,
        'タイプ': building_type,
        '敷金': security_deposit,
        '礼金': key_money,
        '取り扱い店舗': property_stores,
        '取り扱い店舗URL': "https://suumo.jp"+property_stores_link
    })

In [13]:
# データフレームの作成
df = pd.DataFrame(properties_list)

In [14]:
# データフレームの表示（省略せずに全て表示する設定も可能です）
df.head()

,物件名,URL,住所,アクセス1,アクセス2,アクセス3,築年数,賃貸料,管理費,間取り,専有面積,向き,タイプ,敷金,礼金,取り扱い店舗,取り扱い店舗URL
0,Ｍｏｎｋｓｔｏｗｎ夏目坂 104号室,https://suumo.jp/chintai/bc_100375809678/,東京都新宿区戸山１,東京メトロ東西線/早稲田駅 歩4分,都営大江戸線/若松河田駅 歩10分,都営大江戸線/牛込柳町駅 歩12分,新築,28.5万円,管理費 10000円,12LDK,75.09m2,-,マンション,敷28.5万円,礼28.5万円,(株)Street2,https://suumo.jp/chintai/kaisha/kc_030_170304001/
1,レジディア市谷砂土原 Ｓ,https://suumo.jp/chintai/bc_100382140932/,東京都新宿区市谷砂土原町３,都営大江戸線/牛込神楽坂駅 歩6分,東京メトロ東西線/神楽坂駅 歩13分,ＪＲ中央線/市ケ谷駅 歩14分,築17年,115万円,管理費 -,4LDK,180.44m2,南,マンション,敷345万円,礼-,いい部屋ネット大東建託リーシング(株)渋谷店,https://suumo.jp/chintai/kaisha/kc_030_159885726/
2,東京メトロ東西線 落合駅 14階建 築18年,https://suumo.jp/chintai/bc_100382057225/,東京都新宿区上落合１,東京メトロ東西線/落合駅 歩5分,ＪＲ総武線/東中野駅 歩12分,西武新宿線/中井駅 歩11分,築18年,15.5万円,管理費 10000円,2LDK,58.68m2,南,マンション,敷15.5万円,礼15.5万円,(株)ワイエス・ホーム高円寺店,https://suumo.jp/chintai/kaisha/kc_030_082934002/
3,クレストレジデンス東中野,https://suumo.jp/chintai/bc_100382103064/,東京都新宿区北新宿３,ＪＲ中央線/東中野駅 歩4分,ＪＲ山手線/新大久保駅 歩18分,ＪＲ中央線/大久保駅 歩14分,築16年,28万円,管理費 20000円,3LDK,70.19m2,西,マンション,敷28万円,礼28万円,アエラス目白店 (株)アエラス.ER,https://suumo.jp/chintai/kaisha/kc_030_149891011/
4,ＪＲ総武線 東中野駅 5階建 築16年,https://suumo.jp/chintai/bc_100381977764/,東京都新宿区北新宿３,ＪＲ総武線/東中野駅 歩4分,東京メトロ東西線/落合駅 歩11分,東京メトロ丸ノ内線/中野坂上駅 歩16分,築16年,28万円,管理費 20000円,3LDK,70.19m2,西,マンション,敷28万円,礼28万円,(株)東中野不動産,https://suumo.jp/chintai/kaisha/kc_030_179070001/


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   物件名        30 non-null     object
 1   URL        30 non-null     object
 2   住所         30 non-null     object
 3   アクセス1      30 non-null     object
 4   アクセス2      30 non-null     object
 5   アクセス3      29 non-null     object
 6   築年数        30 non-null     object
 7   賃貸料        30 non-null     object
 8   管理費        30 non-null     object
 9   間取り        30 non-null     object
 10  専有面積       30 non-null     object
 11  向き         30 non-null     object
 12  タイプ        30 non-null     object
 13  敷金         30 non-null     object
 14  礼金         30 non-null     object
 15  取り扱い店舗     30 non-null     object
 16  取り扱い店舗URL  30 non-null     object
dtypes: object(17)
memory usage: 4.1+ KB


In [16]:
#DBファイルとして保存する関数
def save_to_database(quotes, db_name=None):
    if db_name is None:
        date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        db_name = f'quotes_{date_str}.db'
    
    try:
        df = pd.DataFrame(quotes)
        conn = sqlite3.connect(db_name)
        df.to_sql('quotes', conn, if_exists='append', index=False)
        conn.close()
        logging.info(f"Data successfully saved to {db_name}")
    except Exception as e:
        logging.error(f"Error saving data to database: {e}")

In [21]:
# データベース名を指定
db_name = 'estate_list.db'  # ここに実際のデータベースファイル名を入力してください

save_to_database(df, db_name)

2024-05-20 00:19:40,116 - INFO - Data successfully saved to estate_list.db
